# FUNDAMENTOS DE ETL COM PYTHON - FERNANDO TIOSSO - DIO

## 1 - Extração de Dados

### Importando a Biblioteca Pandas:

In [ ]:
import pandas as pd # alias comumente utilizado pela comunidade

### Carregar o Arquivo .csv:

In [ ]:
df = pd.read_csv("ocorrencia.csv", sep=",")
# read_csv - função da biblioteca pandas que carrega arquivos .csv
# se o arquivo estiver na mesma pasta que o código, basta informar seu código. Senão, precisará informar o caminho.
# o arquivo será carregado para um "dataframe" quando o carregarmos em uma variável
# com o parâmetro "sep=" informamos qual será o separador das colunas dos dados

In [ ]:
print(df) # exibição do dataframe

### Verificar os Tipos de Dados Carregados:

In [ ]:
df.dtypes
# com o atributo .dtypes do dataframe exibimos os tipos de dados carregados na extração

### Existem funções específicas para trabalhar com datas no Dataframe

#### Acessar apenas a coluna ocorrencia_dia:

In [ ]:
df.ocorrencia_dia
# df - nome do objeto do dataframe
# .ocorrencia_dia - nome da coluna a ser acessada

#### Acessando informações das datas:

In [ ]:
df.ocorrencia_dia.dt.month
# .dt - acessa as funções de data
# .month - retorna o mês da data
# nas condições atuais, retornará um erro, pois os dados não estão no formato de data na coluna.

#### Carregando o Arquivo .csv com as colunas de data no formato correto:

In [ ]:
df = pd.read_csv('ocorrencia.csv', sep=',', parse_dates=['ocorrencia_dia'])
# com o parâmetro "parse_dates=" informamos uma lista com as colunas que deverão ser carregadas com o formato de "data"


In [ ]:
df.dtypes

#### Testando agora a função dt.month deverá retornar os meses das datas:

In [ ]:
df.ocorrencia_dia.dt.month

## 2 - Validação de Dados

### Imprimir as Primeiras 10 linhas do Dataframe:

In [ ]:
df.head(10)
# .head() - imprime as primeiras linhas do dataframe
# (10) - especifica o número de linhas que serão impressas

### Para previnir que os dias e os meses nas datas sejam carregados invertidos, passamos o parâmetro dayfirst=True no carregamento:

In [ ]:
df = pd.read_csv("ocorrencia.csv", sep=",", parse_dates=['ocorrencia_dia'], dayfirst=True)

In [ ]:
df.head(10)

### Verificando as datas finais para conferir o formato:

In [ ]:
df.tail(10)
# .tail() - exibe as últimas linhas do dataframe

### Validação dos Tipos de Dados:

In [ ]:
df.dtypes # lista os tipos de dados das colunas

### Carregando a Biblioteca Pandera:

In [ ]:
import pandera as pa # biblioteca com a qual criamos um padrão para a validação dos dados após eles serem carregados para que possamos detectar ajustes em formatos de dados ou registros divergentes encontrados.

### Criando o Esquema de Validação dos Dados:

In [ ]:
schema = pa.DataFrameSchema(
    columns = {
        "codigo": pa.Column(pa.Int, required=False),
        "codigo_ocorrencia": pa.Column(pa.Int),
        "codigo_ocorrencia2": pa.Column(pa.Int),
        "ocorrencia_classificacao": pa.Column(pa.String),
        "ocorrencia_cidade": pa.Column(pa.String, nullable=True),
        "ocorrencia_uf": pa.Column(pa.String, pa.Check.str_length(2,2), nullable=True),
        "ocorrencia_aerodromo": pa.Column(pa.String, nullable=True),
        "ocorrencia_dia": pa.Column(pa.DateTime, nullable=True),
        "ocorrencia_hora": pa.Column(pa.String, pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9]):([0-5][0-9])?$'), nullable=True),
        "total_recomendacoes": pa.Column(pa.Float, nullable=True)
    }
)

# pa.DataFrameSchema - função que cria o esquema de validação
# columns - descrição das colunas a serem validadas
# pa.Column(pa.Int) - definição do tipo de dados a ser validado na coluna
# schema - variável a qual o esquema é atribuído
# nas colunas onde podem haver valores nulos devemos incluir o parâmetro "nullable=True"
# o parâmetro pa.Check.str_matches(r'^([0-1]?[0-9]|[2][0-3]):([0-5][0-9]):([0-5][0-9])?$') define um padrão de formato de hora a ser validado na coluna "ocorrencia_hora".
# o parâmetro acima pode ser onitido, mas quando explicitado deve ser o segundo a ser descrito
# o parêmetro pa.Check.str_length(2,2) define o tamanho mínimo e máximo dos dados na coluna "ocorrencia_uf" em número de caracteres
# para validarmos um modelo com uma coluna que nem sempre é atualizada, incluímos no esquema da coluna o parâmetro required=False

### Validando o Dataframe com o Esquema criado:

In [ ]:
schema.validate(df)

# validate(df) - função que executação a validação do dataframe "df" segundo o esquema "schema"
# se o dataframe for validado, o dados do dataframe serão exibidos